In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LSTM
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from scipy.signal import find_peaks

Using TensorFlow backend.


In [0]:
data = pd.read_csv("drive/My Drive/JS_newIndicators.csv", index_col= "DateTime")
data.index = pd.to_datetime(data.index)
data = data.drop("pricediff", axis=1)

In [0]:
df = data.copy()

In [0]:
peekidx,_ = find_peaks(df.close.values, prominence = 1.5)#, distance = 5 )
troughidx,_ = find_peaks( - df.close.values, prominence = 1.5)#, distance = 5 )
pk = []
tr = []
pt = []
for i in range(0,len(df)):
  if(i in peekidx):
    pt.append(2)
    pk.append(1)
    tr.append(np.nan)
  elif(i in troughidx):
    pt.append(1)
    tr.append(1)
    pk.append(np.nan)
  else:
    pt.append(0)
    tr.append(np.nan)
    pk.append(np.nan)

In [0]:
df["pt"] = pt
df["pk"] = pk
df["tr"] = tr

In [0]:
peek_future = 5
df["next_price_val"] = df.close.shift(-peek_future)

In [0]:
peek_future = 1
df["next_price_state"] = df.close < df.close.shift(-peek_future)

In [0]:
df.next_price_state.replace(True, 1, inplace = True)
df.next_price_state.replace(False, 0, inplace = True)

In [50]:
df.describe()

,open,high,low,close,volume,sma5,sma10,sma20,emadiff,BB,stochrsifastk,stochrsifastd,ROC,TrueRange,MOM6,MOM12,WilliamR10,WilliamR16,ULOSC,RSI6,RSI12,stochfastkdiff,stochfastddiff,avgPrice,normPrice,sma5diff,sma20diff,smadiff,pricemadiff,pt,pk,tr,next_price_val,next_price_state
count,425579.000000,425579.000000,425579.000000,425579.000000,4.255790e+05,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,4.255790e+05,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,4.255790e+05,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,425579.000000,21074.0,21105.0,425574.000000,425579.000000
mean,640.747307,641.281444,640.206728,640.742790,6.321149e+03,640.746744,640.751700,640.761548,-0.013717,0.006620,49.425182,4.942508e+01,-0.000743,1.100527,-0.011875,-0.023737,-50.968620,-51.120687,49.653348,49.412664,49.551371,0.000139,0.000079,-3.486478e-07,-0.590501,-0.000002,-0.000002,-0.000013,-0.000014,0.148628,1.0,1.0,640.731785,0.457405
std,260.273467,260.455645,260.086887,260.268154,1.726139e+04,260.274047,260.281948,260.297687,1.161761,0.007387,40.918385,3.118212e+01,0.596226,1.256665,2.590106,3.608972,28.716009,27.531508,10.896982,15.783367,11.017909,42.670046,19.098283,1.704587e-03,0.166337,0.000732,0.000354,0.003390,0.003961,0.474948,0.0,0.0,260.249881,0.498183
min,318.700000,319.450000,318.050000,318.850000,0.000000e+00,319.320000,319.765000,320.112500,-20.799548,0.000000,0.000000,-1.302662e-12,-19.912136,0.000000,-86.400000,-88.150000,-100.000000,-100.000000,0.000000,0.000064,0.366216,-100.000000,-33.333333,-1.012658e-01,-0.796223,-0.039778,-0.009866,-0.118029,-0.172417,0.000000,1.0,1.0,318.850000,0.000000
25%,462.000000,462.500000,461.550000,462.000000,7.600000e+02,462.050000,462.090000,462.160000,-0.352072,0.003009,0.000000,2.281203e+01,-0.211359,0.450000,-0.850000,-1.150000,-74.757282,-73.584906,42.290265,39.045988,42.845000,-20.571155,-13.808168,-6.278027e-04,-0.704736,-0.000260,-0.000127,-0.001126,-0.001307,0.000000,1.0,1.0,462.000000,0.000000
50%,551.000000,551.600000,550.500000,551.000000,2.176000e+03,551.060000,551.055000,551.090000,-0.032725,0.004644,48.889648,4.923633e+01,-0.011975,0.800000,-0.050000,-0.050000,-51.388889,-51.612903,49.621926,49.242261,49.344393,0.000000,0.000000,0.000000e+00,-0.647856,-0.000008,-0.000009,-0.000080,-0.000092,0.000000,1.0,1.0,551.000000,0.000000
75%,756.300000,757.000000,755.600000,756.350000,5.918000e+03,756.390000,756.455000,756.523750,0.292836,0.007569,100.000000,7.576916e+01,0.185529,1.300000,0.750000,1.000000,-27.458894,-28.846154,57.039808,59.604102,56.028033,20.153359,13.781671,6.008412e-04,-0.516617,0.000236,0.000110,0.000967,0.001131,0.000000,1.0,1.0,756.350000,1.000000
max,1580.000000,1581.900000,1579.000000,1581.000000,3.647239e+06,1578.530000,1577.580000,1575.917500,16.021948,0.296851,100.000000,1.000000e+02,16.017553,86.950000,57.750000,61.500000,-0.000000,-0.000000,95.862607,99.058419,96.711455,100.000000,33.333333,1.257882e-01,0.010417,0.025293,0.008963,0.090832,0.119889,2.000000,1.0,1.0,1579.650000,1.000000


In [51]:
#################################################################
"""
Model One: Predicting next_price_state with CNN network
"""
model1_data = df.drop(["pt","pk","tr","next_price_val"],axis=1)

model1_Xdata = model1_data.drop("next_price_state", axis = 1).values
model1_ydata = model1_data["next_price_state"].values

model1X = []
model1y = []
look_back = 5
look_back-=1
positive = 0
negative = 0
for i in range(look_back, len(model1_data)):
  if(model1_ydata[i] == 1):
    model1X.append(model1_Xdata[i-look_back:i+1])
    model1y.append(model1_ydata[i])
    positive += 1
    # while(positive < negative):
    #   model1X.append(model1_Xdata[i-look_back:i+1])
    #   model1y.append(model1_ydata[i])
    #   positive += 1
  if(model1_ydata[i] == 0):
    model1X.append(model1_Xdata[i-look_back:i+1])
    model1y.append(model1_ydata[i])
    negative += 1
    # while(positive > negative):
    #   model1X.append(model1_Xdata[i-look_back:i+1])
    #   model1y.append(model1_ydata[i])
    #   negative += 1

print("Positive:",positive)
print("Negative:",negative)

model1X = np.array(model1X)
model1y = np.array(model1y)

print("model1X shape:",model1X.shape)
print("model1y shape:",model1y.shape)

Positive: 194660
Negative: 230915
model1X shape: (425575, 5, 29)
model1y shape: (425575,)


In [0]:
size = 423575
img_rows, img_cols = 5,29
input_shape = (5, 29, 1)
num_classes = 2

#x_train1, x_test1, y_train1, y_test1 = train_test_split(imagesX, imagesY, train_size=0.9, shuffle=False, random_state=116)
x_train1 = model1X[:size]
x_test1 = model1X[size:]

y_train1 = model1y[:size]
y_test1 = model1y[size:]


x_train2, x_test2, y_train2, y_test2 = train_test_split(x_train1, y_train1, train_size=0.9, random_state=116)

x_train3 = np.array(x_train2)
x_train = x_train3.reshape(x_train3.shape[0], img_rows, img_cols, 1)

x_test3 = np.array(x_test2)
x_test = x_test3.reshape(x_test3.shape[0], img_rows, img_cols, 1)


y_train = keras.utils.to_categorical(y_train2, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test2, num_classes=num_classes)


In [0]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(2,2), activation='relu', input_shape=input_shape))
model.add(Conv2D(256, (1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.35))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.50))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['categorical_accuracy'])

In [54]:
model.fit(x_train, y_train,
          batch_size=128, 
          epochs=2, 
          verbose=1,
          validation_data=(x_test, y_test))

Train on 381217 samples, validate on 42358 samples
Epoch 1/2
381217/381217 [==============================] - 794s 2ms/step - loss: 7.3702 - categorical_accuracy: 0.5427 - val_loss: 7.3950 - val_categorical_accuracy: 0.5412
Epoch 2/2
381217/381217 [==============================] - 791s 2ms/step - loss: 7.3702 - categorical_accuracy: 0.5427 - val_loss: 7.3950 - val_categorical_accuracy: 0.5412
